In [ ]:
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import pymysql

db = pymysql.connect(host="localhost", user="rnv_importer", password="rnv_importer", database="rnv_big_data_mining")
cursor = db.cursor()

#vollständig
cursor.execute("SELECT COUNT(*) from stops where (api_plannedDeparture is not null) and (api_realtimeDeparture is not null)")
correct = cursor.fetchone()[0]

#canceled (korrekt)
cursor.execute("SELECT Count(*) from stops inner join journeys ON stops.api_journey = journeys.id where api_canceled = 1")
correct += cursor.fetchone()[0]

#wrong realtime, not canceled
cursor.execute("SELECT Count(*) from stops inner join journeys ON stops.api_journey = journeys.id where api_canceled = 0 and stops.api_realtimeDeparture is null and (api_plannedDeparture is not null)")
wrongrealtime = cursor.fetchone()[0]

#wrong plannedtime
cursor.execute("SELECT COUNT(*) from stops where (api_realtimeDeparture is not null) and (api_plannedDeparture is null)")
wrongplanned = cursor.fetchone()[0]

#all null
cursor.execute("SELECT COUNT(*) from stops where (api_plannedDeparture is null) and (api_realtimeDeparture is null)")
allnull = cursor.fetchone()[0]


df = pd.DataFrame({'Prozentual': [correct, wrongrealtime, wrongplanned, allnull]},
                   index=['korrekt', 'Realtime Null', 'Plannedtime Null', 'keine Zeiten'])

fig, ax = plt.subplots(figsize=(5, 5))

# Funktion zur Generierung des Texts für autopct
def custom_autopct(pct):
    absolute = int(pct / 100 * df.sum().values[0])
    return f'{pct:.1f}%\n({absolute} Fälle)'

# Pie chart mit Prozentangaben, absoluten Zahlen und custom Text
plot = df.plot.pie(y='Prozentual', autopct=custom_autopct, labels=None, ax=ax)
# entfernt Label (links)
ax.set_ylabel('')

# Legende anzeigen
ax.legend(df.index, loc='upper right')

plt.title('Datensatzfehler')
plt.show()